In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from numpy.linalg import inv
from collections import OrderedDict
from operator import itemgetter
import string
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import f1_score
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import time
import cv2 
import cv2 as cv

In [2]:
def loadDataFinal():
    start_time = time.time()
    print 'start...'
    x = np.loadtxt("xP_BB.txt", delimiter=",") # load from text 
    y = np.loadtxt("train_y.csv", delimiter=",") 
    xTest = np.loadtxt("xPTest_BB.txt", delimiter=",")
    x = x.reshape(-1, 32, 32) # reshape 
    xTest = xTest.reshape(-1, 32, 32)
    y = y.reshape(-1, 1) 
    print("Loaded data in --- %s seconds ---" % (time.time() - start_time))
    return [x,y,xTest]
x,y,xTest = loadDataFinal()

start...
Loaded data in --- 58.8411669731 seconds ---


In [3]:
x     = x.astype(np.uint8);
xTest = xTest.astype(np.uint8);
print y.shape
print x.shape
print xTest.shape

(50000, 1)
(50000, 32, 32)
(10000, 32, 32)


## Observations with SVM

In [16]:
def testSVM(x,y,xTest, param_grid):
    start_time = time.time()

    print 'Started...'
        clf = LinearSVC();
    tree = GridSearchCV(clf ,param_grid,n_jobs=-1, refit=True , verbose= 1)
    tree.fit(x,y)
    tree_preds = tree.best_estimator_;
    result = tree_preds.predict(xTest);
    return [tree, result]
    print("--- %s seconds ---" % (time.time() - start_time))

In [19]:
size = 50000
param_grid = {'C':[0.00001,0.0001,0.001]
         }
svm1,svmr1 = testSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print svm1.best_score_
print svm1.best_params_

Started...
Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:   23.5s finished


0.6018
{'C': 1e-05}


In [24]:
size = 50000
param_grid = {'C':[1,50,100]
         }
svm1,svmr1 = testSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print svm1.best_score_
print svm1.best_params_

Started...
Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 14.7min finished


0.54764
{'C': 1}


In [23]:
size = 50000
param_grid = {'C':[0.00001,0.0001,0.001]
              ,'penalty': ['l2']

         }
svm1,svmr1 = testSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print svm1.best_score_
print svm1.best_params_

Started...
Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 13.1min finished


0.73242
{'penalty': 'l2', 'C': 1e-05}


### Effect of size of Dataset

In [5]:
size = 100
clf, result = testSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size]);
print clf.best_score_
print clf.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits
--- 1.23333191872 seconds ---
0.54
{'C': 1e-05, 'random_state': 0}


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    0.8s finished


In [6]:
size = 1000
clf, result = testSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size]);
print clf.best_score_
print clf.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    4.8s finished


--- 6.26605987549 seconds ---
0.625
{'C': 1e-05, 'random_state': 0}


In [7]:
size = 10000
clf, result = testSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size]);
print clf.best_score_
print clf.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  2.4min finished


--- 171.61596489 seconds ---
0.6464
{'C': 1e-05, 'random_state': 0}


In [8]:
size = x.shape[0]
clf, result = testSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size]);
print clf.best_score_
print clf.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 23.1min finished


--- 1649.76570582 seconds ---
0.73266
{'C': 1e-05, 'random_state': 0}


In [9]:
print clf.best_score_
print clf.best_params_

0.73266
{'C': 1e-05, 'random_state': 0}


In [10]:
def generateSubmittionFile(result):
    file = open("submittion.txt","w") 
    file.write("Id,Label\n")
    for i in range(len(result)):
        text = str(i) +','+ str(int(result[i])) + '\n'
        file.write(text)
    print 'Done...'



# Test DecisoinTree

In [11]:
def testDT(x,y,xTest, param_grid):
    start_time = time.time()

    print 'Started...'
    clf = DecisionTreeClassifier();
    tree = GridSearchCV(clf ,param_grid,n_jobs=-1, refit=True , verbose= 1)
    tree.fit(x,y)
    tree_preds = tree.best_estimator_;
    result = tree_preds.predict(xTest);
    return [tree, result]
    print("--- %s seconds ---" % (time.time() - start_time))

In [12]:
size = 50000
param_grid = {#'max_depth': np.arange(20,30,2)
          #,'criterion': ['gini','entropy']
          #,'max_leaf_nodes': np.arange(10,100,10)
          #,"min_samples_split": np.logspace(1.2, 2.0, num=7, dtype=int)
          #,'random_state': np.arange(2,3,1)
         }
tree1,result1 = testDT(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print tree1.best_score_
print tree1.best_estimator_.tree_.max_depth
print tree1.best_estimator_.tree_.n_node_samples

Started...
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   11.5s finished


0.72704
56
[50000  6515  1455 ...     2     1     1]


In [13]:
size = 50000
param_grid = {'max_depth': np.arange(4,60,4)
          ,'criterion': ['gini','entropy']
          #,'max_leaf_nodes': np.arange(10,100,10)
          ,"min_samples_split": [2,4,8,16]
          #,"min_samples_leaf": [1,2]
          #,'random_state': np.arange(2,3,1)
         }
tree2,result2 = testDT(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print tree2.best_score_
print tree2.best_estimator_.tree_.max_depth


Started...
Fitting 3 folds for each of 112 candidates, totalling 336 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 336 out of 336 | elapsed: 17.8min finished


0.74576
12


In [14]:
print tree2.best_estimator_.get_params


<bound method DecisionTreeClassifier.get_params of DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=12,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')>


# Test KNN

In [15]:
def testKNN(x,y,xTest, param_grid):
    clf = KNeighborsClassifier();
    knng = GridSearchCV(clf ,param_grid,n_jobs=-1, refit=True , verbose= 2)
    knng.fit(x,y)
    knn_preds = knng.best_estimator_;
    result = knn_preds.predict(xTest);
    return [knng, result]


In [16]:
param_grid = {'algorithm': ['auto']
          ,'n_neighbors': [5,10,15,20]
          ,"weights": ['uniform','distance']
         }

size = 3000
knn1,result_knn1 = testKNN(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print knn1.best_score_
print knn1.best_params_
print 

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] n_neighbors=5, weights=uniform, algorithm=auto ..................
[CV] n_neighbors=5, weights=uniform, algorithm=auto ..................
[CV] n_neighbors=5, weights=uniform, algorithm=auto ..................
[CV] n_neighbors=5, weights=distance, algorithm=auto .................
[CV] ... n_neighbors=5, weights=uniform, algorithm=auto, total=   3.8s
[CV] n_neighbors=5, weights=distance, algorithm=auto .................
[CV] .. n_neighbors=5, weights=distance, algorithm=auto, total=   3.7s
[CV] n_neighbors=5, weights=distance, algorithm=auto .................
[CV] ... n_neighbors=5, weights=uniform, algorithm=auto, total=   3.8s
[CV] n_neighbors=10, weights=uniform, algorithm=auto .................
[CV] ... n_neighbors=5, weights=uniform, algorithm=auto, total=   4.2s
[CV] n_neighbors=10, weights=uniform, algorithm=auto .................
[CV] .. n_neighbors=5, weights=distance, algorithm=auto, total=   3.8s
[CV] n_neighbors=

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.2min finished


0.8233333333333334
{'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'auto'}



In [17]:
param_grid = {'algorithm': ['auto']
          ,'n_neighbors': [5,7,9]
          ,"weights": ['uniform','distance']
         }

size = 50000
knn2,result_knn2 = testKNN(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print knn1.best_score_
print knn1.best_params_
print  

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] n_neighbors=5, weights=uniform, algorithm=auto ..................
[CV] n_neighbors=5, weights=uniform, algorithm=auto ..................
[CV] n_neighbors=5, weights=uniform, algorithm=auto ..................
[CV] n_neighbors=5, weights=distance, algorithm=auto .................
[CV] ... n_neighbors=5, weights=uniform, algorithm=auto, total=19.4min
[CV] n_neighbors=5, weights=distance, algorithm=auto .................
[CV] ... n_neighbors=5, weights=uniform, algorithm=auto, total=19.9min
[CV] n_neighbors=5, weights=distance, algorithm=auto .................
[CV] .. n_neighbors=5, weights=distance, algorithm=auto, total=19.8min
[CV] n_neighbors=7, weights=uniform, algorithm=auto ..................
[CV] ... n_neighbors=5, weights=uniform, algorithm=auto, total=20.3min
[CV] n_neighbors=7, weights=uniform, algorithm=auto ..................
[CV] .. n_neighbors=5, weights=distance, algorithm=auto, total=17.8min
[CV] n_neighbors=

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 262.7min finished


0.8233333333333334
{'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'auto'}



In [19]:
param_grid = {'algorithm': ['auto']
          ,'n_neighbors': [5,10,15,20]
          ,"weights": ['uniform','distance']
         }

size = 3000
knn1,result_knn1 = testKNN(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print knn1.best_score_
print knn1.best_params_
print 

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] n_neighbors=5, weights=uniform, algorithm=auto ..................
[CV] n_neighbors=5, weights=uniform, algorithm=auto ..................
[CV] n_neighbors=5, weights=uniform, algorithm=auto ..................
[CV] n_neighbors=5, weights=distance, algorithm=auto .................
[CV] ... n_neighbors=5, weights=uniform, algorithm=auto, total=   4.7s
[CV] n_neighbors=5, weights=distance, algorithm=auto .................
[CV] ... n_neighbors=5, weights=uniform, algorithm=auto, total=   5.0s
[CV] n_neighbors=5, weights=distance, algorithm=auto .................
[CV] ... n_neighbors=5, weights=uniform, algorithm=auto, total=   4.6s
[CV] n_neighbors=10, weights=uniform, algorithm=auto .................
[CV] .. n_neighbors=5, weights=distance, algorithm=auto, total=   5.1s
[CV] n_neighbors=10, weights=uniform, algorithm=auto .................
[CV] .. n_neighbors=5, weights=distance, algorithm=auto, total=   5.0s
[CV] n_neighbors=

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.4min finished


0.8233333333333334
{'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'auto'}



# Test Random Forest

In [20]:
def testRandomForest(x,y,xTest, param_grid):
    clf = RandomForestClassifier();
    rf = GridSearchCV(clf ,param_grid,n_jobs=-1, refit=True , verbose= 1)
    rf.fit(x,y)
    rf_preds = rf.best_estimator_;
    result = rf_preds.predict(xTest);
    return [rf, result]


In [21]:
param_grid = { 'criterion': ['gini','entropy']
              ,'max_depth': np.arange(4,60,4)
              ,"min_samples_split": [2,4,8,16]
         }

size = 50000
rf_1,result_rf1 = testRandomForest(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print rf_1.best_score_
print rf_1.best_estimator_

Fitting 3 folds for each of 112 candidates, totalling 336 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 336 out of 336 | elapsed:  4.9min finished


0.87324
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=44, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [22]:
param_grid = { 'criterion': ['gini','entropy']
              ,'max_depth': np.arange(4,40,4)
              , 'max_features': np.arange(1,100,10)
              ,"min_samples_split": [2,4,8,16]
         }
size = 5000
rf_2,result_rf2 = testRandomForest(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print rf_2.best_score_
print rf_2.best_estimator_

Fitting 3 folds for each of 720 candidates, totalling 2160 fits


[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 500 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 1000 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done 1497 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1947 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed:  3.2min finished


0.7884
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=24, max_features=81, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [23]:
param_grid = { 'criterion': ['gini','entropy']
              ,'max_depth': np.arange(20,40,4)
              , 'max_features': np.arange(20,80,10)
              ,"min_samples_split": [2,4,8,16]
         }
size = 50000
rf_2,result_rf2 = testRandomForest(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid);
print rf_2.best_score_
print rf_2.best_estimator_

Fitting 3 folds for each of 240 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 14.3min finished


0.87568
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=24, max_features=60, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [24]:
generateSubmittionFile(result_rf2)

Done...


## Test bagging

In [5]:
def testBagging(x,y,xTest, param_grid, learner):
    clf = BaggingClassifier(learner);
    rf = GridSearchCV(clf ,param_grid,n_jobs=-1, refit=True , verbose= 1)
    rf.fit(x,y)
    rf_preds = rf.best_estimator_;
    result = rf_preds.predict(xTest);
    return [rf, result]

In [26]:
param_grid = { 
         }
size = 10000

learner = KNeighborsClassifier();
bg_1,result_bg1 = testBagging(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid ,learner);
print bg_1.best_score_
print bg_1.best_estimator_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 16.0min finished


0.8626
BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)


In [27]:
param_grid = { 
         }
size = 50000

learner = KNeighborsClassifier();
bg_1,result_bg1 = testBagging(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid ,learner);
print bg_1.best_score_
print bg_1.best_estimator_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 449.2min finished


0.89988
BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)


In [6]:
param_grid = { 'n_estimators': [100]
         }
size = 5000

learner = KNeighborsClassifier();
bg_1,result_bg1 = testBagging(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid ,learner);
print bg_1.best_score_
print bg_1.best_estimator_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 40.1min finished


0.8446
BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=100, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)


In [7]:
param_grid = { 'n_estimators': [100]
         }
size = 50

learner = KNeighborsClassifier();
bg_4,result_bg4 = testBagging(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid ,learner);
print bg_4.best_score_
print bg_4.best_estimator_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/home/mehrzad/Software/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.4s finished


0.38
BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=100, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)


In [8]:
param_grid = { 'n_estimators': [100],
                'max_features': np.arange(20,80,10)
         }
size = 50

learner = KNeighborsClassifier();
bg_5,result_bg5 = testBagging(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid ,learner);
print bg_5.best_score_
print bg_5.best_estimator_

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    1.1s finished


0.42
BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=70,
         max_samples=1.0, n_estimators=100, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)


## Test Boosting


In [9]:
def testBoosting(x,y,xTest, param_grid):
    clf = AdaBoostClassifier()
    rf = GridSearchCV(clf ,param_grid,n_jobs=-1, refit=True , verbose= 1)
    rf.fit(x,y)
    rf_preds = rf.best_estimator_;
    result = rf_preds.predict(xTest);
    return [rf, result]

In [10]:
param_grid = { 'n_estimators': [10]
         }
size = 10000

bo_0,result_bo0 = testBoosting(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid );
print bo_0.best_score_
print bo_0.best_estimator_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.8s finished


0.4509
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=10, random_state=None)


In [11]:
param_grid = { 'n_estimators': [100]
         }
size = 10000

bo_1,result_bo1 = testBoosting(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid );
print bo_1.best_score_
print bo_1.best_estimator_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   17.4s finished


0.614
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)


In [12]:
param_grid = { 'n_estimators': [100]
         }
size = 50000

bo_2,result_bo2 = testBoosting(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid );
print bo_2.best_score_
print bo_2.best_estimator_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.6min finished


0.63228
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)


## Test kernalized SVM

In [13]:
from sklearn.svm import SVC
def testkernelizedSVM(x,y,xTest, param_grid):
    clf = SVC()
    rf = GridSearchCV(clf ,param_grid,n_jobs=-1, refit=True , verbose= 1)
    rf.fit(x,y)
    rf_preds = rf.best_estimator_;
    result = rf_preds.predict(xTest);
    return [rf, result]

In [14]:
param_grid = { 'kernel': ['linear','poly','rbf','sigmoid']
              ,'C':[0.00001,0.0001,0.001]
         }
size = 500

ksvm_1,result_ksvm1 = testkernelizedSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid );
print ksvm_1.best_score_
print ksvm_1.best_estimator_

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    3.1s finished


0.752
SVC(C=1e-05, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [15]:
param_grid = { 'kernel': ['linear','poly','rbf','sigmoid']
              ,'C':[0.00001,0.0001,0.001,0.01,0.1,1]
              ,'degree': [2,3,4,5,6]
         }
size = 5000

ksvm_2,result_ksvm2 = testkernelizedSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid );
print ksvm_2.best_score_
print ksvm_2.best_estimator_

Fitting 3 folds for each of 120 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 34.6min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 63.9min finished


0.8624
SVC(C=1e-05, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [ ]:
param_grid = { 'kernel': ['linear','poly','rbf','sigmoid']
              ,'C':[0.00001,0.0001,0.001]
              ,'degree': [2,3,4,5,6]
         }
size = 50000

ksvm_2,result_ksvm2 = testkernelizedSVM(x.reshape(-1,1024)[:size], y.ravel()[:size], xTest.reshape(-1,1024)[:size],param_grid );
print ksvm_2.best_score_
print ksvm_2.best_estimator_